# 전자우편 텍스트 &rarr; CSV

- [stackoverflow, "Python: Multiple Text Files to Dataframe"](https://stackoverflow.com/questions/44335814/python-multiple-text-files-to-dataframe)

텍스트로 작성된 전자우편 내용에서 필요한 필드만 추출하여 데이터프레임 형태 데이터로 변환시키는 프로젝트를 진행하고 있다.

작업디렉토리(working_directory)


In [9]:
import os
import sys
import re
import csv 

# Take all text files in workingDirectory and put them into a DF.
def convert_email(working_directory, output_directory):
    with open(working_directory +'email_sample.csv', 'w') as csvfile:
      fields = ['Target','Attribute','Label','Time','Full Text'] # fields you're searching for with regex
      csvfield = ['Target','Attribute','Label','Time','Full Text','Filename'] # You want to include the file name in the csv header but not find it with regex
      writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n', fieldnames=fields)
      writer.writeheader() # writes the csvfields list to the header of the csv

      if working_directory == "": working_directory = os.getcwd() + "\\" # Returns current working directory, if workingDirectory is empty.
      i = 0
      for txt in os.listdir(working_directory): # Iterate through text filess in workingDirectory
          print("Processing File: " + str(txt))
          fileExtension = txt.split(".")[-1]
          if fileExtension == "txt":
              textFilename = working_directory + txt # Becomes: \PATH\example.text
              f = open(textFilename,"r")
              data = f.read() # read what is inside

              #print(data) # print to show it is readable
              fieldmatches = {}
              for field in fields:
                regex = "\\s" + field + ":(.*)" # iterates through each of the fields and matches using r"\sTarget:(.*) that selects everything on the line that matches with Target:
                match = re.search(regex, data)
                if match:
                  fieldmatches[field] = match.group(1)
              writer.writerow(fieldmatches) # for each file creates a dict of fields and their values and then adds that row to the csv
              i += 1 # counter
      print("Successfully read " + str(i) + " files.")


working_directory = "data/email/" # Put your source directory of text files here
output_directory  = "data/email/" # Put your source directory of text files here

convert_email(working_directory, output_directory)



Processing File: .DS_Store
Processing File: email_sample.txt
Processing File: email_sample.csv
Successfully read 1 files.


# 전자우편 


전자우편 데이터가 아래와 같은 형식으로 쭉 연결되어 있다고 가정하다.
이러한 전자우편 텍스트 데이터에서 정규표현식을 동원하여 텍스트 일부를 추출하여 데이터프레임 형태로 변형시켜보자.

## 전자우편 데이터셋

In [24]:
# To: source@collab.sakaiproject.org
# From: stephen.marquard@uct.ac.za
# Subject: [sakai] svn commit: r39772 - content/branches/sakai_2-5-x/content-impl/impl/src/java/org/sakaiproject/content/impl
# X-Content-Type-Outer-Envelope: text/plain; charset=UTF-8
# X-Content-Type-Message-Body: text/plain; charset=UTF-8
# Content-Type: text/plain; charset=UTF-8
# X-DSPAM-Result: Innocent
# X-DSPAM-Processed: Sat Jan  5 09:14:16 2008
# X-DSPAM-Confidence: 0.8475
# X-DSPAM-Probability: 0.0000

## 전자우편 데이터 전처리

working_directory = "data/email/"
file_name = 'mbox-short.txt'

file_path = working_directory + file_name

email_dict = dict()

file_handler = open(file_path)

for line in file_handler:
    if not line.startswith('From: '): continue
    pieces = line.split() 
    email = pieces[1]
    email_dict[email] = email_dict.get(email, 0) + 1

print(email_dict)